## Introduction:

Extracting codes from standard terminologies is a regular and indispensable task often encountered in medical and healthcare fields. Diagnosis codes, procedure codes, cancer site and morphology codes are all manually extracted from patient records by trained human coders. There are servrals use cases of extracted codes, for example: 
- Insurance billing and reimbursement 
- Medical facilities quality control 
- Scientific work such as Epidemiological studies 
- Cohort identification for clinical trials. 

In this repoort using combination of unsupervised and supervised machine learning we will try to extracting international classification of diseases, clinical modification, 9th revision (ICD-9-CM) diagnosis codes from electronic health records (EHRs). 

The dataset contains de-identified EHRs containing both diagnostic and procedural ICD codes, but we will focus on extracting only the most common diagnostic ICD codes. The methods and approach used in this report are general and it can also be applied to other medical code extraction tasks.

In this notebook, we will explore the supervised machine learning portion.
##### Motivation

Our goal for supervised machine learning portion is to process features extracted from the relevant portion of the patient's Electronic Health Record (EHR) and predict relevant diagnostic ICD codes related to the EHR diagnosis. As more than one code can be assigned to the same EHR diagnosis, the supervised learning approach to use is Multi-Label Classification, where each ICD code is treated as individual label. There are many real life examples of Multi Label classifications, such categorizing movies by genere, classifying news articles, labeling objects from image classifications, etc. 

Multi Label classification probel can be often confused with Multi Class classification problem. Multi Class classification assumes that classes are mutually exclusive. Multi label classification treats each label as a different classification problem. What we have here is a binary Multi Label problem, where we try to identify if each label i.e. the ICD-9 code is appropriate or not appropriate for the EHR diagnostic summary of the patient. 
We wish to present this as a solution to aid the human medical coder in assigning correct ICDs as a decision support application. 


In [176]:
import pandas as pd
import numpy as np
import re, time
import pickle
from collections import Counter

In [169]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import Normalizer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import f1_score, accuracy_score, hamming_loss, roc_auc_score 
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from skmultilearn.problem_transform import BinaryRelevance
import xgboost as xgb
import joblib

## Data Location Declarations

We have used MIMIC-III dataset for training and validating our models. MIMIC-III is an open dataset of medical records of ICU stays of deidentified patients at Beth-Israel Hospital. It has EHRs of 40000 individuals, data contained in it is deidentified. More information about the data and how to access it is given in the initial Data section of the report which is common for both unsupervised and supervised learning. 

Here we will only discuss about the data manipulation and relevant information extraction which is relevant for the supervised portion of the report. 

In [32]:
#Location of your data directory
datadir = "Data/"
#Location of your model directory
mdldir = "Models/"
# Make sure to have following files in the data directory:
# 1. DIAGNOSES_ICD.csv : Assigned diagnostic ICDs for each EHR
diagicd_file = "DIAGNOSES_ICD.csv"
# 2. df_cleaned_wt_prob_topic_dist.pkl : Preprocessed DataFrame
dfpkl_file = "df_cleaned_wt_prob_topic_dist.pkl"



## Functions

In [163]:
def diag_token_extract(text):
    # Function takes string input of EHR text
    # Extracts "Discharge Diagnosis" portion of the full note
    # and returns extracted word tokens
    rawdiags = re.findall(r"(?<=Diagnos.s:\n)((.+\n)*)(?=)", text)
    rawdiags = [x[0].split('\n')[:-1] for x in rawdiags]
    findiags = [diag.lower().split() for diags in rawdiags for diag in diags if diag != 'Primary Diagnosis:']
    fltdiags = [diag for sublist in findiags for diag in sublist]
    alfdiags = [item for item in fltdiags if item.isalpha()]
    return alfdiags

#_____________________________________________________________
# Insert missing "." in ICD codes
def icd_dot(initcode, isdiagcode=False):
    # Function reformats ICD codes present in the raw data
    # from abcxy format to abc.xyz format
    initcode = list(str(initcode))
    icdcode = initcode[:]
    if isdiagcode==False:
        icdcode.insert(2,'.')
    else:
        if initcode[0]=='E':
            if len(initcode)>4:
                icdcode.insert(4,'.')
        else:
            if len(initcode)>3:
                icdcode.insert(3,'.')
        icdcode=icdcode[:5] # Regroup ICDs at abc.x level by stripping yz codes sections
    return "".join(icdcode)

#_____________________________________________________________
def onelevelup(codes,topicds):
    # Function takes string consisting of ICD 9 codes (abc.xyz format)
    # Reformats them to abc.x format to improve avg.codes/document density
    # Outputs a set of reformatted codes
    codes = codes.split(",")
    setofcodes = {''.join(list(code)[:5]) for code in codes if code in topicds}
    return list(setofcodes)#', '.join(setofcodes)
#_____________________________________________________________
def dataprep(df, diagicd_df, mostcommon=50):
    # Use only if unsupervised Topic Modeling exercise is not done 
    # and topic vectors are not present in the saved pickle file
    # Restructuring ICD code format 
    diagicd_df['ICD_CODE']=diagicd_df['ICD9_CODE'].apply(lambda x: icd_dot(x,isdiagcode=True))
    topicds=Counter(diagicd_df.ICD_CODE).most_common(mostcommon)
    topicds = [icd[0] for icd in topicds] # Most common ICDs in descending order as list
    df['ICDset'] = df['ICDs'].apply(lambda x: onelevelup(x, topicds)) # Format ICD to abc.x format
    df['Tokens'] = df['TEXT'].apply(lambda x: diag_token_extract(x)) # Extract word tokens

    df = df[df['Tokens'].map(lambda d: len(d)) > 0] # Drop rows with no tokens
    df = df[df['ICDset'].map(lambda d: len(d)) > 0] # Drop rows with no labels i.e. ICD codes

    print("Shape of the dataset = ",df.shape)
    print("Number of Unique set of ICDs = ", df['ICDset'].astype(str).nunique())

    # Transforming prediction target 'y' to multilabel matrix 'y_'
    y = df['ICDset']
    mlb = MultiLabelBinarizer()
    y_ = mlb.fit_transform(y)

    x_train, x_test, y_train, y_test = train_test_split(df['Tokens'],
                                              y_, test_size=0.2)

    return x_train, x_test, y_train, y_test
#_____________________________________________________________
def dataprep_topmdlfeat(df, diagicd_df, mostcommon=50, numtopics=20):
    # Function
    # Restructuring ICD code format 
    diagicd_df['ICD_CODE']=diagicd_df['ICD9_CODE'].apply(lambda x: icd_dot(x,isdiagcode=True))
    topicds=Counter(diagicd_df.ICD_CODE).most_common(mostcommon)
    topicds = [icd[0] for icd in topicds] # Most common ICDs in descending order as list
    df['ICDset'] = df['ICDs'].apply(lambda x: onelevelup(x, topicds)) # Format ICD to abc.x format
    df['Tokens'] = df['TEXT'].apply(lambda x: diag_token_extract(x)) # Extract word tokens

    df = df[df['Tokens'].map(lambda d: len(d)) > 0] # Drop rows with no tokens
    df = df[df['ICDset'].map(lambda d: len(d)) > 0] # Drop rows with no labels i.e. ICD codes

    print("Shape of the dataset = ",df.shape)
    print("Number of Unique set of ICDs = ", df['ICDset'].astype(str).nunique())

    # Expanding topic model distribution vector to columns
    global topiclist
    topiclist = ['topic_'+ str(i) for i in range(numtopics)]
    df[topiclist] = pd.DataFrame(df.prob_dist_topics.tolist(), index= df.index)
    
    # Transforming prediction target 'y' to multilabel matrix 'y_'
    y = df['ICDset']
    global mlb
    mlb = MultiLabelBinarizer()
    y_ = mlb.fit_transform(y)

    x_train, x_test, y_train, y_test = train_test_split(df[topiclist+['Tokens']],
                                              y_, test_size=0.2)

    return x_train, x_test, y_train, y_test
#______________________________________________________________________
def humaneval(y_test,y_pred,id=111):
    print("Human assigned ICDs - ", mlb.inverse_transform(y_test)[id])
    print("Model assigned ICDs - ",mlb.inverse_transform(y_pred)[id])
    print("Correctly assigned ICDs - ",[i for i in mlb.inverse_transform(y_pred)[id] if i in mlb.inverse_transform(y_test)[id]])
    print('Features extracted from Discharge Diagnosis: ', X_test['Tokens'].iloc[id])
    return


## Load Data

In [5]:
# Load prepared dataframe
df = pd.read_pickle(datadir+dfpkl_file)
diagicd_df = pd.read_csv(datadir+diagicd_file)
# Loading mdlresults to keep track of models and 
try:
    mdlresults_df.head(1)
except: 
    mdlresults_df = pd.read_pickle(mdldir+'mdlresults_df.pkl')

In [149]:
# Split data into Training and Test sets
mc, num = 100, 20 # Dataset for #'mc' most common ICDs, num = number of topics from topic modeling
X_train, X_test, y_train, y_test = dataprep_topmdlfeat(df, diagicd_df, mostcommon=mc, numtopics=num)

Shape of the dataset =  (36783, 9)
Number of Unique set of ICDs =  26618


## Pipeline: Methods, Hyperparameters and Metrics

### Methods - 
Most of the machine learning classifiers are built for single target classification problems i.e. for a  single label with binary class values. So we transformed a multi label classification problem to multiple single label problems using the following techniques. ScikitLearn’s One-Vs-Rest classifier and MultiOutputClassifier. In our observations both of the classifiers performed similarly and hence we decided to go with One-Vs-Rest classifier for our final model.

Once, a multi label classification problem is transformed to multiple single label classification problem, we experimented with following classifiers for single label classification -  
- Logistic Regression 
- Support Vector Machines (SVM) 
- XGBoost 
(Note: In XGBoost 1.6, it has experimental support for multi-output regression and multi-label classification with Python package. More details for the same can be found here. [[https://xgboost.readthedocs.io/en/stable/tutorials/multioutput.html]].) 
Feature Tuning: 

We then trained models using TF-IDF word vectors as features with and without topic vectors obtained from topic modeling done in the unsupervised learning section.
We also experimented with following features and compared different models’ performance - 

For tf-idf vectorizer:  
max_df : We experimented with max_df in the range of .95 to 0.75 to ignore words higher than the set threshold.  
Max_features : Top 1000, 10000, 25000 and 50000 word tokens ordered by term frequency  
  
For classifiers:  
Solver: We tried different solvers for the Logistic Regression classifier, and picked ‘sag’ solver for our final model  
max_iter : We experimented with 100 (default), 1000, 2500, and 5000 max_iter limits to help the solver converge on a solution. 




In [187]:
# TFIDF helper function for pipeline
def identity_tokenizer(text):
        return text

def model_pipeline_evals(max_df = 0.8, max_features = 50000, max_iter=2500, estimator_ind=0, classifier_ind=0, topicvec = 0):
    # Function to pick estimator, classifier and topicvector as additional feature
    # estimator_ind, classifier_ind = 0, 0 #default vals
    # Function outputs, predicted labels for test dataset and the supervised learning model

    tfidf_vec = TfidfVectorizer(tokenizer=identity_tokenizer, lowercase=False, 
            max_df=max_df, max_features=max_features)

    # Model selection
    estimators_dict = {
                        'lr':LogisticRegression(max_iter=max_iter, solver='sag', multi_class='ovr'), #, class_weight='balanced'
                        'svm': LinearSVC(max_iter=max_iter, penalty='l2'),
                        'xgb': xgb.XGBClassifier()
                    }
    estimators_list = list(estimators_dict.keys())
    estimator = estimators_dict[estimators_list[estimator_ind]] 
    # estimator = estimators_dict[estimators_list[estimator_ind]] #default

    # Multilabel classification have to be converted to a binary problem. 
    # The two most common conversions are:
    #           OvO - One vs One
    #           OvR - One vs Rest  
    classifier_dict = {
                        'ovr': OneVsRestClassifier(estimator),
                        'moc': MultiOutputClassifier(estimator, n_jobs=-1),
                        'xgb': xgb.XGBClassifier(tree_method="hist")
                    }
    classifier_list = list(classifier_dict.keys())
    classifier = classifier_dict[classifier_list[classifier_ind]]

    if topicvec == 0:
        # Pipeline for supervised learning without Topic Vectors
        pipeline = Pipeline(
            [
                ("tfidf", tfidf_vec),
                ("clf", classifier), 
            ]
        )
        start_time = time.time()
        mdl = pipeline.fit(X_train['Tokens'], y_train)
        elapsed_time = time.time() - start_time
        y_pred = mdl.predict(X_test['Tokens'])

    else:
    # Pipeline for supervised learning with Topic Vectors
        preprocess = ColumnTransformer([
                ('distribution_norm', StandardScaler(), topiclist),
                ("tfidf", tfidf_vec, "Tokens")
            ],remainder="passthrough"
        )

        pipeline = Pipeline(
            [
                ("preprocess", preprocess),
                ("clf", classifier),
            ]
        )
        start_time = time.time() 
        mdl = pipeline.fit(X_train, y_train) # mdl fit
        elapsed_time = time.time() - start_time
        y_pred = mdl.predict(X_test)

    precision, recall, _ = precision_recall_curve(
        y_test.ravel(), y_pred.ravel()
    )
    mdlname = estimators_list[estimator_ind]+'_'+classifier_list[classifier_ind]+'_'+str(mc)+'_'+str(topicvec)+'.pkl'
    joblib.dump(mdl, mdldir+mdlname) #pickle.dump(mdl, open(mdldir+mdlname, 'wb'))
    print("Trained Model :", mdlname, "Training time :",'{0:.2f}'.format(elapsed_time))

    score = [
                mdlname,
                topicvec,
                f1_score(y_test, y_pred, average="micro"),
                accuracy_score(y_test, y_pred),
                average_precision_score(y_test, y_pred, average="micro"),
                roc_auc_score(y_test, y_pred, average="micro"),
                hamming_loss(y_test, y_pred),
                '{0:.2f}'.format(elapsed_time),
                precision,
                recall
            ]
    mdlresults_df.loc[len(mdlresults_df.index)] = score
    return y_pred, mdl


##### Model Evaluation Metrics Results


The code below will train the models to predict user chosen number for most common ICDs in the dataset, and save the models as pickle file and save the performance evaluation dataframe also as a pickle file in the Models folder. The performance of model is evaluated over 5 randomly generated sets of Training and Test data, and we will finally see the average performance of each model in the resulting dataframe. 


Uncomment the code below if you wish to retrain the models and evaluate its performance.

In [ ]:
# for mc in [50, 100, 250]:
#     num = 20
#     for _ in range(5): # Repeat 5 times to get average of evaluation metrics
#         X_train, X_test, y_train, y_test = dataprep_topmdlfeat(df, diagicd_df, mostcommon=mc, numtopics=num)
#         for i in range(3):
#             for j in range(2):
#                 model_pipeline_evals(max_df=0.8, max_features=50000, max_iter=2500, estimator_ind=i, classifier_ind=0, topicvec = j)
# mdlresults_df.to_pickle("mdlresults_df")


##### Logging history

In [ ]:
"""###### Logging history:
Shape of the dataset =  (34774, 9)
Number of Unique set of ICDs =  9616
Trained Model : lr_ovr_50_0.pkl Training time : 4.59
Trained Model : lr_ovr_50_1.pkl Training time : 11.98
Trained Model : svm_ovr_50_0.pkl Training time : 1.57
Trained Model : svm_ovr_50_1.pkl Training time : 93.89
Trained Model : xgb_ovr_50_0.pkl Training time : 45.29
Trained Model : xgb_ovr_50_1.pkl Training time : 90.90
Shape of the dataset =  (34774, 9)
Number of Unique set of ICDs =  9616
Trained Model : lr_ovr_50_0.pkl Training time : 4.32
Trained Model : lr_ovr_50_1.pkl Training time : 12.43
Trained Model : svm_ovr_50_0.pkl Training time : 4.25
Trained Model : svm_ovr_50_1.pkl Training time : 95.44
Trained Model : xgb_ovr_50_0.pkl Training time : 49.63
Trained Model : xgb_ovr_50_1.pkl Training time : 90.97
Shape of the dataset =  (34774, 9)
Number of Unique set of ICDs =  9616
Trained Model : lr_ovr_50_0.pkl Training time : 4.54
Trained Model : lr_ovr_50_1.pkl Training time : 12.81
Trained Model : svm_ovr_50_0.pkl Training time : 1.64
Trained Model : svm_ovr_50_1.pkl Training time : 87.48
Trained Model : xgb_ovr_50_0.pkl Training time : 44.16
Trained Model : xgb_ovr_50_1.pkl Training time : 90.86
Shape of the dataset =  (34774, 9)
Number of Unique set of ICDs =  9616
Trained Model : lr_ovr_50_0.pkl Training time : 5.49
Trained Model : lr_ovr_50_1.pkl Training time : 12.87
Trained Model : svm_ovr_50_0.pkl Training time : 1.55
Trained Model : svm_ovr_50_1.pkl Training time : 94.03
Trained Model : xgb_ovr_50_0.pkl Training time : 47.13
Trained Model : xgb_ovr_50_1.pkl Training time : 95.25
Shape of the dataset =  (34774, 9)
Number of Unique set of ICDs =  9616
Trained Model : lr_ovr_50_0.pkl Training time : 4.81
Trained Model : lr_ovr_50_1.pkl Training time : 13.61
Trained Model : svm_ovr_50_0.pkl Training time : 1.56
Trained Model : svm_ovr_50_1.pkl Training time : 101.40
Trained Model : xgb_ovr_50_0.pkl Training time : 46.76
Trained Model : xgb_ovr_50_1.pkl Training time : 93.99
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : lr_ovr_100_0.pkl Training time : 8.30
Trained Model : lr_ovr_100_1.pkl Training time : 25.71
Trained Model : svm_ovr_100_0.pkl Training time : 3.87
Trained Model : svm_ovr_100_1.pkl Training time : 143.56
Trained Model : xgb_ovr_100_0.pkl Training time : 94.78
Trained Model : xgb_ovr_100_1.pkl Training time : 194.13
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : xgb_ovr_100_1.pkl Training time : 193.96
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : lr_ovr_100_0.pkl Training time : 8.64
Trained Model : lr_ovr_100_1.pkl Training time : 23.41
Trained Model : svm_ovr_100_0.pkl Training time : 2.68
Trained Model : svm_ovr_100_1.pkl Training time : 131.33
Trained Model : xgb_ovr_100_0.pkl Training time : 88.41
Trained Model : xgb_ovr_100_1.pkl Training time : 205.23
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : lr_ovr_100_0.pkl Training time : 8.06
Trained Model : lr_ovr_100_1.pkl Training time : 24.45
Trained Model : svm_ovr_100_0.pkl Training time : 2.74
Trained Model : svm_ovr_100_1.pkl Training time : 147.86
Trained Model : xgb_ovr_100_0.pkl Training time : 90.81
Trained Model : xgb_ovr_100_1.pkl Training time : 213.17
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : lr_ovr_100_0.pkl Training time : 8.64
Trained Model : lr_ovr_100_1.pkl Training time : 23.41
Trained Model : svm_ovr_100_0.pkl Training time : 2.68
Trained Model : svm_ovr_100_1.pkl Training time : 131.33
Trained Model : xgb_ovr_100_0.pkl Training time : 88.41
Trained Model : xgb_ovr_100_1.pkl Training time : 205.23
Shape of the dataset =  (35837, 9)
Number of Unique set of ICDs =  19547
Trained Model : lr_ovr_100_0.pkl Training time : 8.06
Trained Model : lr_ovr_100_1.pkl Training time : 24.45
Trained Model : svm_ovr_100_0.pkl Training time : 2.74
Trained Model : svm_ovr_100_1.pkl Training time : 147.86
Trained Model : xgb_ovr_100_0.pkl Training time : 90.81
Trained Model : xgb_ovr_100_1.pkl Training time : 213.17
Shape of the dataset =  (36975, 9)
Number of Unique set of ICDs =  28172
Trained Model : lr_ovr_250_0.pkl Training time : 18.89
Trained Model : lr_ovr_250_1.pkl Training time : 56.17
Trained Model : svm_ovr_250_0.pkl Training time : 8.03
Trained Model : svm_ovr_250_1.pkl Training time : 225.57
Trained Model : xgb_ovr_250_0.pkl Training time : 233.72
Trained Model : xgb_ovr_250_1.pkl Training time : 494.59
Shape of the dataset =  (36975, 9)
Number of Unique set of ICDs =  28172
Trained Model : lr_ovr_250_0.pkl Training time : 17.82
Trained Model : lr_ovr_250_1.pkl Training time : 48.28
Trained Model : svm_ovr_250_0.pkl Training time : 7.24
Trained Model : svm_ovr_250_1.pkl Training time : 213.72
Trained Model : xgb_ovr_250_0.pkl Training time : 305.20
Trained Model : xgb_ovr_250_1.pkl Training time : 481.65
Shape of the dataset =  (36975, 9)
Number of Unique set of ICDs =  28172
Trained Model : lr_ovr_250_0.pkl Training time : 21.68
Trained Model : lr_ovr_250_1.pkl Training time : 85.77
Trained Model : svm_ovr_250_0.pkl Training time : 10.16
Trained Model : svm_ovr_250_1.pkl Training time : 332.21
Trained Model : xgb_ovr_250_0.pkl Training time : 248.13
"""

## Human Evaluation of results

In [193]:
mdlresults_df

,name,topicvectorused,f1,accuracy,avg_precision,roc_auc,hammingloss,mdlfittime,prici_array,recall_array
0,lr_ovr_50_0.pkl,0,0.359289,0.065277,0.259948,0.613742,0.101653,4.66,"[0.11829895481944368, 0.7062208824335434, 1.0]","[1.0, 0.24093118922961854, 0.0]"
1,lr_ovr_50_1.pkl,1,0.360493,0.064845,0.260323,0.614288,0.101653,12.35,"[0.11829895481944368, 0.7047062023939065, 1.0]","[1.0, 0.24219334330590875, 0.0]"
2,lr_ovr_50_0.pkl,0,0.359289,0.065277,0.259948,0.613742,0.101653,5.25,"[0.11829895481944368, 0.7062208824335434, 1.0]","[1.0, 0.24093118922961854, 0.0]"
3,lr_ovr_50_1.pkl,1,0.360493,0.064845,0.260323,0.614288,0.101653,11.74,"[0.11829895481944368, 0.7047062023939065, 1.0]","[1.0, 0.24219334330590875, 0.0]"
4,lr_ovr_50_0.pkl,0,0.359289,0.065277,0.259948,0.613742,0.101653,4.21,"[0.11829895481944368, 0.7062208824335434, 1.0]","[1.0, 0.24093118922961854, 0.0]"
...,...,...,...,...,...,...,...,...,...,...
81,xgb_ovr_250_0.pkl,0,0.313415,0.025828,0.172133,0.599202,0.034023,248.13,"[0.03847034333690033, 0.7006436737438755, 1.0]","[1.0, 0.20185441461389428, 0.0]"
82,xgb_ovr_250_1.pkl,1,0.313678,0.024476,0.172127,0.599343,0.034032,495.99,"[0.03847034333690033, 0.6996168582375479, 1.0]","[1.0, 0.20215887074453362, 0.0]"
83,lr_ovr_250_0.pkl,0,0.269046,0.019878,0.149422,0.581697,0.034633,23.56,"[0.038362800998759536, 0.7068130830086197, 1.0]","[1.0, 0.16614393960420773, 0.0]"
84,lr_ovr_250_1.pkl,1,0.269548,0.019743,0.149665,0.581888,0.034625,48.32,"[0.038362800998759536, 0.7067137809187279, 1.0]","[1.0, 0.16653251547364623, 0.0]"


In [ ]:
# Aggregate cross validatation results of trained models
avg_mdlresults_df = pd.read_pickle("avg_mdlresults_df")
avg_mdlresults_df = avg_mdlresults_df.sort_values('name')
avg_mdlresults_df

In [181]:
# Load a pretrained model from the Models directory :
# We have trained models for 3 different classifiers namely LogisticRegression (lr), SVM (svm) and XGBoost (xgb)
# Models are trained for predicting 50, 100 and 250 most commonly used ICDs, and the number will be reflected in the name
# Please select appropriate value of variable "mostcommon" based on most common ICDs to predict i.e. 50, 100 or 250

id=999
num = 20 # Number of topics from topic vector obtained from topic modeling 
X_train, X_test, y_train, y_test = dataprep_topmdlfeat(df, diagicd_df, mostcommon=250, numtopics=num)

In [ ]:
mdl1 = joblib.load('Models/lr_ovr_250_1.pkl')
humaneval(y_test,mdl1.predict(X_test),id) 

In [ ]:
mdl1 = joblib.load('Models/xgb_ovr_250_1.pkl')
humaneval(y_test,mdl1.predict(X_test),id)

#### Saved example of human evaluation of model performance and discussions

In [164]:
# Model - Logistic Regression, One Vs Rest, 200 most common ICDs
# id=8
# humaneval(y_test,y_pred,id)


Human assigned ICDs -  ('272.4', '285.1', '287.5', '424.1')
Model assigned ICDs -  ('272.4', '401.9', '424.1')
Correctly assigned ICDs -  ['272.4', '424.1']
Features extracted from Discharge Diagnosis:  ['coronary', 'artery', 'disease', 'gerd', 'hyperlipidemia', 'aortic', 'stenosis', 'obesity', 'cataracts']


In the above example, the human encoder has assigned ICDs -
- '272.4' = Other and unspecified hyperlipidemia
- '285.1' = Acute posthemorrhagic anemia
- '287.5' = Thrombocytopenia, unspecified
- '424.1' = Aortic valve disorders

Features extracted from Discharge Diagnosis:  ['coronary', 'artery', 'disease', 'gerd', 'hyperlipidemia', 'aortic', 'stenosis', 'obesity', 'cataracts']

It seems that our model was able to correctly pick codes based on relevant words like 'coronary', 'artery', 'disease','hyperlipidemia', 'aortic', 'stenosis' but failed to pick Thrombocytopenia which is a condition in which you have a low blood platelet count, and Acute posthemorrhagic anemia is a condition that develops when you lose a large amount of blood quickly. The failure for picking these could be blamed on not extracting all the relevant information from the NOTESEVENTS.csv which contains the full Electronic Health Record of the patient. 

In [165]:
# # Model - Logistic Regression, One Vs Rest, 200 most common ICDs
# id=555
# humaneval(y_test,y_pred,id)


Human assigned ICDs -  ('272.0', 'V17.3')
Model assigned ICDs -  ('401.9', '424.0')
Correctly assigned ICDs -  []
Features extracted from Discharge Diagnosis:  ['mitral', 'valve', 'coronary', 'artery']


In the above example, the human encoder has assigned ICDs -
- '272.4' = Pure hypercholesterolemia 
- 'V17.3' = Family history of ischemic heart disease

It seems our model failed to pick heart related terms such as 'valve', 'coronary', 'artery', but the context here is family history of heart dieseases and that was not present in the portion of the text that was extracted for the sample. We believe that incorporating more relevant sections of the discharge note might improve the model prediction. 

### Ideas for future work:

For unsupervised learning supplement the unsupervised dataset with long description of code as EHR diagnosis note and pass the single ICD code as a label, to gauge how well the ICD code is attributed to different topics identified by the model. ICD-9 CM codes are broadly categorized in 20 categories. We can compare if the model congregates related codes in correct clusters or not. Improved topic models can lead to better topic vectors which have already shown to help improve the performance of the supervised learning model.

Explore use of Top2vec model for topic modeling task.

Filtering the entire note based on the SNOMED-CT medical terms dictionary.

Considering extracting bi-grams and ni-grams as features.

Add code recommendation explainability which highlights which parts of the diagnosis note are relevant for the recommended code

We are currently predicting codes in the format "xyz.a" (complete format is xyz.abc). We will test the model prediction for codes in the format "xyz"   i.e. at the category level, and in the format "xyz.ab" and "xyz.abc"

State of the art current research on auto labeling of ICDs show that deep neural networks with attention have significantly improved the prediction performance. We will train a LSTM model and explore scope for further improvement using deep learning methods. 

Explore if the learning from this project/model be used to improve/implement template for free text inputs from human operators.

Auto complete suggestions to human operators while they input patient notes in the system.


### Ethics Discussion: 

- Since the ICD codes are used for insurance billing and reimbursement purposes, incorrect ICD if used as wrongly suggested by the model then it may cause monetary harm to the individuals or institutions.
- Incorrect labeling of ICDs may harm any scientific research, clinical trials done based on such records.
- General rule among human medical codes is that the most appropriate ICD among set of relevant ICDs is to be used; the model in current state lacks the awareness to assess that, and we as the model developers lack the domain expertise to make that judgment. 
- For insurance purposes, some codes may result in higher insurance payouts than other comparable codes, which may introduce bias towards such predictions from the model if it is present in the underlying data. We have to be cognizant of this with our model predictions.

Hence the current model is prone to making mistakes and it should not be deployed as it is and more work is needed given the potential harm listed above. We believe that significant human in the loop validation work is required.